# Sampling

A recurrence Neural Network can be used as a Generative model once it was trained. Currently this is a common practice not only to study how well a model has learned a problem, but to learn more about the problem domain itself. In fact, this approach is being used for music generation and composition.

The process of generation is explained in the picture below:

<img src="local/imgs/dinos3.png" style="width:500;height:300px;">
<caption><center> **Figure **: In this picture, we assume the model is already trained. We pass in $x^{\langle 1\rangle} = \vec{0}$ at the first time step, and have the network then sample one character at a time. </center></caption>

Let's do an example:

In [1]:
import sys
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Masking
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
import nltk
nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to /home/julian/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [3]:
# load ascii text and covert to lowercase
raw_text = nltk.corpus.gutenberg.raw('bible-kjv.txt')

In [5]:
raw_text[100:1000]

'Genesis\n\n\n1:1 In the beginning God created the heaven and the earth.\n\n1:2 And the earth was without form, and void; and darkness was upon\nthe face of the deep. And the Spirit of God moved upon the face of the\nwaters.\n\n1:3 And God said, Let there be light: and there was light.\n\n1:4 And God saw the light, that it was good: and God divided the light\nfrom the darkness.\n\n1:5 And God called the light Day, and the darkness he called Night.\nAnd the evening and the morning were the first day.\n\n1:6 And God said, Let there be a firmament in the midst of the waters,\nand let it divide the waters from the waters.\n\n1:7 And God made the firmament, and divided the waters which were\nunder the firmament from the waters which were above the firmament:\nand it was so.\n\n1:8 And God called the firmament Heaven. And the evening and the\nmorning were the second day.\n\n1:9 And God said, Let the waters under the heav'

In [4]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [5]:
char_to_int

{'\n': 0,
 ' ': 1,
 '!': 2,
 "'": 3,
 '(': 4,
 ')': 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '0': 9,
 '1': 10,
 '2': 11,
 '3': 12,
 '4': 13,
 '5': 14,
 '6': 15,
 '7': 16,
 '8': 17,
 '9': 18,
 ':': 19,
 ';': 20,
 '?': 21,
 'A': 22,
 'B': 23,
 'C': 24,
 'D': 25,
 'E': 26,
 'F': 27,
 'G': 28,
 'H': 29,
 'I': 30,
 'J': 31,
 'K': 32,
 'L': 33,
 'M': 34,
 'N': 35,
 'O': 36,
 'P': 37,
 'Q': 38,
 'R': 39,
 'S': 40,
 'T': 41,
 'U': 42,
 'V': 43,
 'W': 44,
 'Y': 45,
 'Z': 46,
 '[': 47,
 ']': 48,
 'a': 49,
 'b': 50,
 'c': 51,
 'd': 52,
 'e': 53,
 'f': 54,
 'g': 55,
 'h': 56,
 'i': 57,
 'j': 58,
 'k': 59,
 'l': 60,
 'm': 61,
 'n': 62,
 'o': 63,
 'p': 64,
 'q': 65,
 'r': 66,
 's': 67,
 't': 68,
 'u': 69,
 'v': 70,
 'w': 71,
 'x': 72,
 'y': 73,
 'z': 74}

In [6]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  4332554
Total Vocab:  75


To train the model we are going to use sequences of 60 characters and because of the data set is too large, we are going to use only the firs 200000 sequences.

In [7]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 60
dataX = []
dataY = []
n_chars = 200000
for i in range(0, n_chars - seq_length, 3):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  66647


In [8]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [9]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [9]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

Note that the entire dataset is used for training

In [13]:
model.fit(X, y, epochs=20, batch_size=128)

Train on 66647 samples
Epoch 1/20
66647/66647 [==============================] - 40s 598us/sample - loss: 3.1200
Epoch 2/20
66647/66647 [==============================] - 39s 584us/sample - loss: 2.7512
Epoch 3/20
66647/66647 [==============================] - 39s 587us/sample - loss: 2.6322
Epoch 4/20
66647/66647 [==============================] - 39s 586us/sample - loss: 2.5703
Epoch 5/20
66647/66647 [==============================] - 39s 588us/sample - loss: 2.5130
Epoch 6/20
66647/66647 [==============================] - 39s 584us/sample - loss: 2.4685
Epoch 7/20
66647/66647 [==============================] - 39s 591us/sample - loss: 2.4290
Epoch 8/20
66647/66647 [==============================] - 40s 599us/sample - loss: 2.3925
Epoch 9/20
66647/66647 [==============================] - 39s 585us/sample - loss: 2.3600
Epoch 10/20
66647/66647 [==============================] - 39s 586us/sample - loss: 2.3289
Epoch 11/20
66647/66647 [==============================] - 41s 616us/sample 

In [14]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [18]:
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
"  thirty there.

18:31 And he said, Behold now, I have taken  "
thee  and tee came of the fardhr of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the land of the l

### The result is not what we expected mainly because of three resons:

- The model requires to be trained with a larger dataset in order to better capture the dynamics of the language.
- During validation it is not recommendable to select the output with maximum probability but to use the output distribution as parameters to sample from a multinomial distribution. This avoid the model to get stuck in a loop.
- A more flexible model with more data could get better results. 

In [11]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

## Using a more complex model with the whole dataset

This problem is complex computationally speaking, so the next model was run using GPU. The LSTM layers were replaced by CuDNNLSTM that are suitable to GPU training. These layers were removed in the new alpha version of tensor flow to improve compatibility.

In [19]:
#from tensorflow.compat.v1.keras.layers import CuDNNLSTM

In [12]:
batch_size = 128
model = Sequential()
model.add(LSTM(256, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(rate=0.2))
model.add(LSTM(256))
model.add(Dropout(rate=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

More data could produce memory erros so we have to create a data_generator function for the problem:

In [23]:
class KerasBatchGenerator(object):

    def __init__(self, data, num_steps, batch_size, vocabulary, skip_step=1):
        self.data = data
        self.num_steps = num_steps
        self.batch_size = batch_size
        self.vocabulary = vocabulary
        # this will track the progress of the batches sequentially through the
        # data set - once the data reaches the end of the data set it will reset
        # back to zero
        self.current_idx = 0
        # skip_step is the number of words which will be skipped before the next
        # batch is skimmed from the data set
        self.skip_step = skip_step
        
    def generate(self):
        x = np.zeros((self.batch_size, self.num_steps, 1))
        y = np.zeros((self.batch_size, self.vocabulary))
        while True:
            for i in range(self.batch_size):
                if self.current_idx + self.num_steps >= len(self.data):
                    # reset the index back to the start of the data set
                    self.current_idx = 0
                seq_in = self.data[self.current_idx:self.current_idx + self.num_steps]
                x[i, :, 0] = np.array([char_to_int[char] for char in seq_in])/ float(n_vocab)
                seq_out = self.data[self.current_idx + self.num_steps]
                temp_y = char_to_int[seq_out]
                # convert all of temp_y into a one hot representation
                y[i, :] = np_utils.to_categorical(temp_y, num_classes=self.vocabulary)
                self.current_idx += self.skip_step
            yield x, y

In [25]:
batch_size = 128
train_data_generator = KerasBatchGenerator(raw_text, seq_length, batch_size, n_vocab, skip_step=3)

In [13]:
#model.fit_generator(train_data_generator.generate(), epochs=30, steps_per_epoch=n_chars/batch_size)
model.fit(X, y, epochs=10, batch_size=128)

Train on 66647 samples
Epoch 1/10
66647/66647 [==============================] - 93s 1ms/sample - loss: 1.7100
Epoch 2/10
66647/66647 [==============================] - 92s 1ms/sample - loss: 1.6891
Epoch 3/10
66647/66647 [==============================] - 92s 1ms/sample - loss: 1.6603
Epoch 4/10
66647/66647 [==============================] - 93s 1ms/sample - loss: 1.6373
Epoch 5/10
66647/66647 [==============================] - 92s 1ms/sample - loss: 1.6124
Epoch 6/10
66647/66647 [==============================] - 93s 1ms/sample - loss: 1.5917
Epoch 7/10
66647/66647 [==============================] - 93s 1ms/sample - loss: 1.5699
Epoch 8/10
66647/66647 [==============================] - 93s 1ms/sample - loss: 1.5489
Epoch 9/10
66647/66647 [==============================] - 93s 1ms/sample - loss: 1.5303
Epoch 10/10
66647/66647 [==============================] - 93s 1ms/sample - loss: 1.5049


As we saw in previous classes, the model trained using batch_input_shape requires a similar batch for validation, so in order to evaluate the model using a single sequence, we have to create a new model with a batch_size = 1 and pass on the learnt weights of the first model to the new one. 

In [14]:
# re-define the batch size
n_batch = 1
# re-define model
new_model = Sequential()
new_model.add(LSTM(256, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), return_sequences=True))
new_model.add(Dropout(rate=0.2))
new_model.add(LSTM(256))
new_model.add(Dropout(rate=0.2))
new_model.add(Dense(y.shape[1], activation='softmax'))
# copy weights
old_weights = model.get_weights()
new_model.set_weights(old_weights)
# compile model
new_model.compile(loss='categorical_crossentropy', optimizer='adam')

In [18]:
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = np.reshape(pattern, (1, seq_length, 1))
    x = x / float(n_vocab)
    prediction = new_model.predict(x, verbose=0)[0]
    index = sample(prediction, 0.3)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" ith pitch.

6:15 And this is the fashion which thou shalt ma "
ke the sooe of the pinetern of the ling of the gart of the earth, and they hived his father so booe? and the sealle teie unto him, There dre thee a pat tn mege mi the land of Eanaan, and tooo his sons, what we well mot to taee in the farth, and ther ho the lonn of the land of the earth,
and the called the cayi of the ming of the earth, and tpoo his sons, and his boethren shat were brrughted and fathhen,

26:26 And the caysle were toro the earth, and wpon him woto the earth, and the dalled his father shatl be boessed thee.

25:14 And the said snto him with thee erom the farth, and spor the land of Eoy said uhe daysh of the earth, and tpoo his father s seie. 
27:14 And the said unto the men of the creak of the gart of the gard of Egypt, and ho the land of the earth of the earth, and the soaee of the gialt were hir dathhters,
and said, Sher seall the sors of the mlngnn of the darth with his sones.

25:24 And the said s

In [110]:
# serialize model to JSON
model_json = new_model.to_json()
with open("modelgenCuDNNLSTM.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
new_model.save_weights("modelgenCuDNNLSTM.h5")
print("Saved model to disk")

Saved model to disk
